# 11. Đào tạo Deep Neural Networks
Trong Chương 10, chúng tôi đã giới thiệu các mạng nơ-ron nhân tạo và đào tạo các mạng nơ-ron sâu đầu tiên của chúng tôi. Nhưng chúng là những mạng lưới rất nông, chỉ có một vài lớp ẩn. Điều gì sẽ xảy ra nếu bạn cần giải quyết một vấn đề rất phức tạp, chẳng hạn như phát hiện hàng trăm loại đối tượng trong hình ảnh có độ phân giải cao? Bạn có thể cần phải đào tạo một DNN sâu hơn nhiều, có lẽ với 10 lớp hoặc nhiều hơn, mỗi lớp chứa hàng trăm tế bào thần kinh, được kết nối bởi hàng trăm nghìn kết nối. Đây sẽ không phải là một cuộc đi dạo trong công viên:
- Đầu tiên, bạn sẽ phải đối mặt với vấn đề tiêu biến gradient(hoặc bùng nổ gradient) ảnh hưởng đến các mạng thần kinh sâu và khiến các lớp thấp hơn rất khó đào tạo.
- Thứ hai, bạn có thể không có đủ dữ liệu đào tạo cho một mạng lớn như vậy hoặc có thể quá tốn kém để gắn nhãn.
- Thứ ba, đào tạo có thể cực kỳ chậm.
- Thứ tư, một mô hình với hàng triệu tham số sẽ có nguy cơ bị trang bị quá mức cho tập huấn luyện, đặc biệt nếu không có đủ các trường hợp huấn luyện hoặc chúng quá nhiễu.

## 11.1 Tiêu Biến/Bùng nỗ Gradients (Vanishing/Exploding Gradients)
Như chúng ta đã thảo luận trong Chương 10, thuật toán lan truyền ngược hoạt động bằng cách đi từ lớp đầu ra đến lớp đầu vào, lan truyền gradient lỗi trên đường đi. Khi thuật toán đã tính toán gradient của hàm chi phí liên quan đến từng tham số trong mạng, nó sử dụng các gradient này để cập nhật từng tham số với bước Gradient Descent.

Thật không may, gradient thường ngày càng nhỏ hơn khi thuật toán tiến dần xuống các lớp thấp hơn. Do đó, bản cập nhật Gradient Descent để lại trọng lượng kết nối lớp dưới hầu như không thay đổi và việc đào tạo không bao giờ tập trung vào một giải pháp tốt. Đây được gọi là vấn đề gradient biến mất. Trong một số trường hợp, điều ngược lại có thể xảy ra: các gradient có thể ngày càng lớn hơn, vì vậy nhiều lớp nhận được cập nhật trọng lượng cực lớn và thuật toán phân kỳ. Đây là vấn đề chuyển sắc bùng nổ, mà hầu hết đều gặp phải trong các mạng nơ-ron tuần hoàn. Nói một cách tổng quát hơn, các mạng nơ-ron sâu có độ dốc không ổn định; các lớp khác nhau có thể học ở nhiều tốc độ khác nhau.

Mặc dù hành vi đáng tiếc này đã được quan sát thực nghiệm trong một thời gian khá dài (đó là một trong những lý do tại sao mạng nơ-ron sâu hầu như bị bỏ rơi trong một thời gian dài), nhưng chỉ đến khoảng năm 2010, việc hiểu nó mới đạt được nhiều tiến bộ đáng kể. Một bài báo có tiêu đề “*Understanding the Difficulty of Training Deep Feedforward Neural Networks*”của Xavier Glorot và Yoshua Bengio1 đã tìm thấy một số nghi vấn, bao gồm sự kết hợp của chức năng kích hoạt sigmoid hậu cần phổ biến và kỹ thuật khởi tạo trọng số phổ biến nhất vào thời điểm đó, cụ thể là khởi tạo ngẫu nhiên bằng cách sử dụng phân phối chuẩn với giá trị trung bình là 0 và độ lệch chuẩn là 1. Tóm lại, họ đã chỉ ra rằng với hàm kích hoạt và sơ đồ khởi tạo này, phương sai của các đầu ra của mỗi lớp lớn hơn nhiều so với phương sai của các đầu vào của nó. Tiếp tục trong mạng, phương sai tiếp tục tăng sau mỗi lớp cho đến khi hàm kích hoạt bão hòa ở các lớp trên cùng. Điều này thực sự trở nên tồi tệ hơn bởi thực tế là hàm logistic có trung bình là 0,5, không phải 0 (hàm tiếp tuyến hyperbol có trung bình là 0 và hoạt động tốt hơn một chút so với hàm logistic trong các mạng sâu)

Nhìn vào hàm kích hoạt logistic (xem Hình 11-1), bạn có thể thấy rằng khi các đầu vào trở nên lớn (âm hoặc dương), hàm bão hòa ở 0 hoặc 1, với đạo hàm cực kỳ gần bằng 0. Vì vậy, khi quá trình lan truyền ngược bắt đầu, nó hầu như không có gradient để truyền ngược lại qua mạng và những gì gradient nhỏ tồn tại tiếp tục bị loãng khi quá trình lan truyền ngược tiến triển xuống các lớp trên cùng, vì vậy thực sự không còn gì cho các lớp dưới.

![](images/11.1.png)
*Hình 11.1 Hình 11-1. Bão hòa hàm kích hoạt logistic*

### Khởi tạo Glorot và He
Trong bài báo của họ, Glorot và Bengio đề xuất một cách để giảm bớt đáng kể vấn đề này. Chúng ta cần tín hiệu xử lý đúng theo cả hai hướng: theo hướng thuận khi đưa ra dự đoán và theo hướng ngược lại khi truyền ngược gradient. Chúng tôi không muốn tín hiệu bị tắt, cũng như không muốn nó bùng nổ và bão hòa. Để tín hiệu đi đúng hướng, các tác giả lập luận rằng chúng ta cần phương sai của các đầu ra của mỗi lớp bằng phương sai của các đầu vào của nó, và chúng ta cũng cần các gradient có phương sai bằng nhau trước và sau khi truyền qua một lớp trong hướng ngược lại. Thực tế không thể đảm bảo cả hai trừ khi lớp có số lượng đầu vào và tế bào thần kinh bằng nhau (những con số này được gọi là *fan-in* và *fan-out* của lớp), nhưng họ đã đề xuất một giải pháp tốt đã được chứng minh là hoạt động rất tốt. trong thực tế: trọng số kết nối của mỗi lớp phải được khởi tạo ngẫu nhiên như được mô tả trong Công thức 11-1, trong đó $f an_{avg} = (f an_{in} + f an_{out}) / 2$. Chiến lược khởi tạo này được gọi là khởi tạo Xavier (theo tên tác giả) hoặc khởi tạo Glorot (theo họ của anh ta)

*Phương trình 11.1 Khởi tạo Glorot (khi sử dụng logistic activation function)*

- Phân phối chuẩn với giá trị trung bình 0 và phương sai $ σ^2 = \frac{1}{fan_{avg}}$
- Hoặc phân phối đồng đều giữa $−r$ và $+ r$, với $r = \sqrt{\frac{3}{fan_{avg}}}$

*Bảng 11-1. Các tham số khởi tạo cho từng hàm kích hoạt*

| Initialization | Activation functions  	| $σ²$ (Normal) 	|
|-	|-	|-	|-	|
| Glorot  	| None, Tanh, Logistic, Softmax |$1/fan_{avg}$  	| 
| He  | ReLU & variants 	| $2 / fan_{in}$  	|
| LeCun  	|  SELU  	| $1 / fan_{in} $	|

Theo mặc định, Keras sử dụng khởi tạo Glorot với phân phối diều. Bạn có thể thay đổi điều này thành Khởi tạo He bằng cách đặt `kernel_initializer = "he_uniform"` hoặc `kernel_initializer = "he_normal"` khi tạo một lớp, như sau:

In [ ]:
keras.layers.Dense(10, activation='relu', kernel_initializer="he_normal")

Nếu bạn muốn khởi tạo He với phân phối đều, nhưng dựa trên $fan_{avg}$ thay vì $fan_{in}$, bạn có thể sử dụng trình khởi tạo `VarianceScaling` như sau:

In [ ]:
he_avg_init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg', distribution='uniform')
keras.layers.Dense(10, activation="sigmoid", kernel_initializer=he_avg_init)

## Các hàm kích hoạt không bão hòa
Một trong những hiểu biết sâu sắc trong bài báo năm 2010 của Glorot và Bengio là các vấn đề về gradient biến mất / bùng nổ một phần là do lựa chọn sai chức năng kích hoạt. Cho đến lúc đó, hầu hết mọi người đều cho rằng nếu Mẹ Thiên nhiên đã chọn sử dụng các chức năng kích hoạt đại khái sigmoid trong các tế bào thần kinh sinh học, thì chúng hẳn là một lựa chọn tuyệt vời. Nhưng nó chỉ ra rằng các chức năng kích hoạt khác hoạt động tốt hơn nhiều trong mạng nơ-ron sâu, đặc biệt là chức năng kích hoạt ReLU, chủ yếu là vì nó không bão hòa các giá trị dương (và cũng vì nó khá nhanh để tính toán).

Thật không may, chức năng kích hoạt ReLU không hoàn hảo. Nó gặp phải một vấn đề được gọi là ReLU đang chết: trong quá trình huấn luyện, một số nơ-ron thực sự chết, có nghĩa là chúng ngừng xuất ra bất kỳ thứ gì khác ngoài 0. Trong một số trường hợp, bạn có thể thấy rằng một nửa số nơ-ron trong mạng của bạn đã chết, đặc biệt nếu bạn sử dụng learning rate lớn. Một nơron chết khi trọng số của nó được điều chỉnh theo cách mà tổng trọng số của các đầu vào của nó là âm đối với tất cả các trường hợp trong tập huấn luyện. Khi điều này xảy ra, nó chỉ tiếp tục xuất ra các số 0 và sự sụt giảm gradient không ảnh hưởng đến nó nữa vì gradient của hàm ReLU là 0 khi đầu vào của nó là âm.

Để giải quyết vấn đề này, bạn có thể muốn sử dụng một biến thể của hàm ReLU, chẳng hạn như ReLU bị rò rỉ. Hàm này được định nghĩa là $LeakyReLU_α (z) = max (αz, z)$ (xem Hình 11-2). Siêu tham số α xác định mức độ "rò rỉ" của hàm: nó là độ dốc của hàm đối với $z <0$ và thường được đặt thành 0,01. Độ dốc nhỏ này đảm bảo rằng các ReLU bị rò rỉ không bao giờ chết; họ có thể hôn mê kéo dài, nhưng cuối cùng họ vẫn có cơ hội tỉnh lại. Một bài báo năm 2015 đã so sánh một số biến thể của chức năng kích hoạt ReLU và một trong những kết luận của nó là các biến thể rò rỉ luôn hoạt động tốt hơn chức năng kích hoạt ReLU nghiêm ngặt. Trên thực tế, đặt $α = 0,2$ (rò rỉ lớn) dường như mang lại hiệu suất tốt hơn so với $α = 0,01$ (rò rỉ nhỏ). Họ cũng đánh giá ReLU bị rò rỉ ngẫu nhiên (RReLU), trong đó α được chọn ngẫu nhiên trong một phạm vi nhất định trong quá trình đào tạo và nó được cố định thành giá trị trung bình trong quá trình thử nghiệm. Nó cũng hoạt động khá tốt và dường như hoạt động như một bộ điều chỉnh (giảm nguy cơ trang bị quá mức bộ đào tạo).

![](images/11.2.png)

*Hình 11.2 Leaky ReLU*

Cuối cùng nhưng không kém phần quan trọng, một bài báo năm 2015 của Djork-Arné Clevert et al.6 đã đề xuất một hàm kích hoạt mới được gọi là đơn vị tuyến tính hàm mũ (exponential linear unit-ELU) hoạt động tốt hơn tất cả các biến thể ReLU trong các thử nghiệm của họ: thời gian đào tạo được giảm xuống và mạng nơ-ron hoạt động tốt hơn trên bộ thử nghiệm. Nó được biểu diễn trong Hình 11-3 và Công thức 11-2 cho thấy định nghĩa của nó.

*Phương trình 11-2. ELU activation function*

![](images/E11.2.png)

*Hình 11.3 ELU activation function*

![](images/11.3.png)

Nó trông rất giống với chức năng ReLU, với một vài điểm khác biệt chính:
- Đầu tiên, nó nhận các giá trị âm khi $z <0$, cho phép đơn vị có đầu ra trung bình gần bằng 0. Điều này giúp giảm bớt vấn đề gradient biến mất, như đã thảo luận trước đó. Siêu tham số $α$ xác định giá trị mà hàm ELU tiếp cận khi $z$ là một số âm lớn. Nó thường được đặt thành 1, nhưng bạn có thể điều chỉnh nó giống như bất kỳ siêu thông số nào khác nếu bạn muốn.
- Thứ hai, nó có một gradient khác không đối với $z <0$, điều này tránh được vấn đề tế bào thần kinh chết.
- Thứ ba, nếu α bằng 1 thì hàm trơn tru ở mọi nơi, kể cả xung quanh $z = 0$, điều này giúp tăng tốc độ Gradient Descent, vì nó không bị nảy nhiều sang trái và phải của $z = 0$.

Hạn chế chính của hàm kích hoạt ELU là tính toán chậm hơn so với ReLU và các biến thể của nó (do sử dụng hàm mũ), nhưng trong quá trình đào tạo, điều này được bù đắp bằng tốc độ hội tụ nhanh hơn. Tuy nhiên, tại thời điểm thử nghiệm, mạng ELU sẽ chậm hơn mạng ReLU.

Để sử dụng chức năng kích hoạt ReLU bị rò rỉ, bạn phải tạo một cá thể LeakyReLU như sau:

In [ ]:
leaky_relu = keras.layers.LeakyReLU(alpha=0.2)
layer = keras.layers.Dense(10, activation=leaky_relu, kernel_initializer="he_normal")

Đối với PReLU, chỉ cần thay thế `LeakyRelu (alpha = 0,2)` bằng `PReLU()`. Hiện tại không có chính thức triển khai `RReLU` trong Keras, nhưng bạn có thể khá dễ dàng triển khai `RReLU` của riêng mình (xem các bài tập ở cuối Chương 12).

Để kích hoạt SELU, chỉ cần đặt `activation = "selu"` và `kernel_initializer="lecun_normal"` khi tạo một lớp:

In [ ]:
layer = keras.layers.Dense(10, activation="selu", kernel_initializer="lecun_normal")

### Batch Normalization
Mặc dù việc sử dụng khởi tạo He cùng với ELU (hoặc bất kỳ biến thể nào của ReLU) có thể giảm đáng kể các vấn đề về tiêu biến gradient / bùng nổ khi bắt đầu đào tạo, nhưng điều đó không đảm bảo rằng chúng sẽ không quay lại trong quá trình đào tạo.

Trong một bài báo năm 2015 ("*Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift*"), Sergey Ioffe và Christian Szegedy đã đề xuất một kỹ thuật gọi là **Batch Normalization** (BN) để giải quyết các vấn đề về gradient biến mất / bùng nổ. Kỹ thuật này bao gồm thêm một hoạt động trong mô hình ngay trước hoặc sau hàm kích hoạt của mỗi lớp ẩn, chỉ cần căn giữa và chuẩn hóa từng đầu vào, sau đó chia tỷ lệ và dịch chuyển kết quả bằng cách sử dụng hai vectơ tham số mới trên mỗi lớp: một để mở rộng quy mô, khác để chuyển dịch. Nói cách khác, thao tác này cho phép mô hình tìm hiểu quy mô và giá trị trung bình tối ưu của từng đầu vào của lớp. Trong nhiều trường hợp, nếu bạn thêm lớp BN làm lớp đầu tiên của mạng nơ-ron của mình, bạn không cần phải chuẩn hóa tập huấn luyện của mình (ví dụ: sử dụng StandardScaler): lớp BN sẽ làm điều đó cho bạn (tốt, xấp xỉ, vì nó chỉ xem xét từng đợt một và nó cũng có thể bán lại và thay đổi từng tính năng đầu vào).

Để căn giữa và chuẩn hóa các đầu vào, thuật toán cần ước tính giá trị trung bình và độ lệch chuẩn của mỗi đầu vào. Nó thực hiện như vậy bằng cách đánh giá giá trị trung bình và độ lệch chuẩn của mỗi đầu vào so với mini-batch hiện tại (do đó có tên là Batch Normalization (Chuẩn hóa hàng loạt). Toàn bộ hoạt động được tóm tắt trong Công thức 11-3.

*Phương trình 11-3. Thuật toán Batch Normalization *

![](images/E11.3.png)

- $μ_B$ là vectơ của trung bình đầu vào, được đánh giá trên toàn bộ mini-batch B (nó chứa một giá trị trung bình cho mỗi đầu vào)
- $σ_B$ là vectơ của độ lệch chuẩn đầu vào, cũng được đánh giá trên toàn bộ minibatch (nó chứa một độ lệch chuẩn cho mỗi đầu vào).
- $m_B$ là số lượng cá thể trong mini-batch.
- $\hat{X}^{(i)}$ là vector zero-centerd và chuẩn hóa đầu vào cho mỗi $i$.
- $γ$ là vectơ tham số tỷ lệ đầu ra cho lớp (nó chứa một thông số tỷ lệ cho mỗi đầu vào)
- $⊗$ đại diện cho phép nhân theo phần tử (mỗi đầu vào được nhân với tham số tỷ lệ đầu ra tương ứng của nó).
- $β$ là vector tham số dịch chuyển đầu ra (offset) cho lớp (nó chứa một tham số offset cho mỗi đầu vào). Mỗi đầu vào được bù đắp bởi tham số dịch chuyển tương ứng của nó.
- $ϵ$ là một số nhỏ để tránh chia cho số 0 (thường là $10^{–5}$). Đây được gọi là thuật ngữ làm mịn.
- $z^{(i)}$ là đầu ra của hoạt động BN: nó là phiên bản được thay đổi tỷ lệ và dịch chuyển của các đầu vào.

Vì vậy, trong quá trình đào tạo, BN chỉ cần chuẩn hóa đầu vào của mình sau đó thay đổi tỷ lệ và hiệu chỉnh chúng. Tốt! Còn lúc kiểm tra thì sao? Vâng, nó không phải là đơn giản. Thật vậy, chúng tôi có thể cần đưa ra dự đoán cho các trường hợp riêng lẻ hơn là cho một loạt trường hợp: trong trường hợp này, chúng tôi sẽ không có cách nào để tính toán trung bình và độ lệch chuẩn của từng đầu vào.
Hơn nữa, ngay cả khi chúng ta có một loạt cá thể, nó có thể quá nhỏ hoặc các cá thể có thể không độc lập và được phân phối giống nhau (independent and identically distributed -IID), do đó, số liệu thống kê tính toán trên các cá thể hàng loạt sẽ không đáng tin cậy (trong quá trình đào tạo, các lô không nên quá nhỏ, nếu có thể hơn 30 trường hợp, và tất cả các trường hợp phải là IID, như chúng ta đã thấy trong Chương 4). Một giải pháp có thể là đợi cho đến khi kết thúc huấn luyện, sau đó chạy toàn bộ tập huấn luyện thông qua mạng nơron và tính toán giá trị trung bình và độ lệch chuẩn của từng đầu vào của lớp BN. Sau đó, các phương tiện đầu vào “cuối cùng” và độ lệch chuẩn này có thể được sử dụng thay cho các phương tiện đầu vào theo lô và độ lệch chuẩn khi đưa ra dự đoán. Tuy nhiên, người ta thường ưu tiên ước tính các thống kê cuối cùng này trong quá trình đào tạo bằng cách sử dụng giá trị trung bình động của phương tiện đầu vào và độ lệch chuẩn của lớp. Tóm lại, bốn vectơ tham số được học trong mỗi lớp chuẩn hóa hàng loạt: γ (vectơ tỷ lệ ouput) và β (vectơ bù đầu ra) được học thông qua truyền ngược thông thường, và μ (vectơ trung bình đầu vào cuối cùng) và σ (cuối cùng vectơ độ lệch chuẩn đầu vào) được ước tính bằng cách sử dụng đường trung bình động hàm mũ. Lưu ý rằng μ và σ được ước lượng trong quá trình đào tạo, nhưng chúng hoàn toàn không được sử dụng trong quá trình đào tạo, chỉ sau khi đào tạo (để thay thế các phương tiện đầu vào theo lô và độ lệch chuẩn trong Công thức 11-3).

Các tác giả đã chứng minh rằng kỹ thuật này đã cải thiện đáng kể tất cả các mạng nơ-ron sâu mà họ đã thử nghiệm, dẫn đến cải thiện đáng kể trong nhiệm vụ phân loại ImageNet (ImageNet là một cơ sở dữ liệu lớn về hình ảnh được phân loại thành nhiều lớp và thường được sử dụng để đánh giá hệ thống thị giác máy tính). Vấn đề độ dốc biến mất đã được giảm thiểu mạnh mẽ, đến mức họ có thể sử dụng các chức năng kích hoạt bão hòa như tanh và thậm chí là chức năng kích hoạt logistic. Các mạng cũng ít nhạy cảm hơn nhiều với việc khởi tạo trọng số. Họ đã có thể sử dụng tỷ lệ học tập lớn hơn nhiều, tăng tốc đáng kể quá trình học tập.

Tuy nhiên, Batch Normalization sẽ thêm một số phức tạp vào mô hình (mặc dù nó có thể loại bỏ nhu cầu chuẩn hóa dữ liệu đầu vào, như chúng ta đã thảo luận trước đó). Hơn nữa, có một hình phạt về thời gian chạy: mạng nơ-ron đưa ra các dự đoán chậm hơn do các tính toán bổ sung được yêu cầu ở mỗi lớp. Vì vậy, nếu bạn cần dự đoán nhanh như chớp, bạn có thể muốn kiểm tra xem quá trình khởi tạo ELU + He đơn giản như thế nào trước khi sử dụng Batch Normalization.

#### Thực hiện Batch Normalization với Keras
Như với hầu hết mọi thứ với Keras, việc triển khai Batch Normalization khá đơn giản. Chỉ cần thêm lớp `BatchNormalization` trước hoặc sau mỗi chức năng kích hoạt của lớp ẩn và tùy chọn thêm lớp BN cũng như lớp đầu tiên trong mô hình của bạn. Ví dụ: mô hình này áp dụng BN sau mỗi lớp ẩn và là lớp đầu tiên trong mô hình (sau khi làm phẳng các hình ảnh đầu vào):

In [3]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])

Trong ví dụ nhỏ này chỉ với hai lớp ẩn, không chắc rằng Chuẩn hóa hàng loạt sẽ có tác động rất tích cực, nhưng đối với các mạng sâu hơn, nó có thể tạo ra sự khác biệt to lớn. 

Hãy phóng to một chút. Nếu hiển thị tóm tắt mô hình, bạn có thể thấy rằng mỗi lớp BN thêm 4 tham số cho mỗi đầu vào: $γ, β, μ$ và $σ$ (ví dụ: lớp BN đầu tiên thêm 3136 tham số, gấp 4 lần 784). Hai tham số cuối cùng, $μ$ và $σ$, là các đường trung bình động, chúng không bị ảnh hưởng bởi sự lan truyền ngược, vì vậy Keras gọi chúng là “Không thể truyền” (nếu bạn đếm tổng số tham số BN, 3136 + 1200 + 400 và chia cho hai , bạn nhận được 2.368, là tổng số các thông số không thể đào tạo trong mô hình này).

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
batch_normalization_2 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_3 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

Hãy xem các thông số của lớp BN đầu tiên. Hai là có thể đào tạo (bằng backprop) và hai là không:

In [5]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization_1/gamma:0', True),
 ('batch_normalization_1/beta:0', True),
 ('batch_normalization_1/moving_mean:0', False),
 ('batch_normalization_1/moving_variance:0', False)]

Bây giờ khi bạn tạo một lớp BN trong Keras, nó cũng tạo ra hai hoạt động sẽ được gọi bởi Keras ở mỗi lần lặp trong quá trình đào tạo. Các hoạt động này sẽ cập nhật các đường trung bình động.

In [6]:
model.layers[1].updates

[]

Các tác giả của bài báo về BN đã lập luận ủng hộ việc thêm các lớp BN trước các chức năng kích hoạt, thay vì sau (như chúng tôi vừa làm). Có một số tranh luận về điều này, vì nó dường như phụ thuộc vào nhiệm vụ. Vì vậy, đó là một điều nữa bạn có thể thử nghiệm để xem tùy chọn nào hoạt động tốt nhất trên tập dữ liệu của bạn. Để thêm các lớp BN trước các chức năng kích hoạt, chúng ta phải xóa chức năng kích hoạt khỏi các lớp ẩn và thêm chúng dưới dạng các lớp riêng biệt sau các lớp BN. Hơn nữa, vì lớp Chuẩn hóa hàng loạt bao gồm một tham số bù đắp cho mỗi đầu vào, bạn có thể xóa cụm từ thiên vị khỏi lớp trước (chỉ cần chuyển `use_bias = False` khi tạo nó):

```
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
    keras.layers.Activation("elu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])
```

Lớp `BatchNormalization` có khá nhiều siêu tham số mà bạn có thể tinh chỉnh. Các giá trị mặc định thường sẽ ổn, nhưng đôi khi bạn có thể cần phải điều chỉnh `momentum` (động lượng). Siêu tham số này được sử dụng khi cập nhật đường trung bình động hàm mũ: được cung cấp một giá trị mới $v$ (nghĩa là vectơ mới của phương tiện đầu vào hoặc độ lệch chuẩn được tính theo batch hiện tại), giá trị trung bình đang chạy được cập nhật bằng cách sử dụng phương trình sau:

$\hat{v}$ &#8592 $\hat{v} × momentum + v × (1 − momentum)$

Giá trị momentum tốt thường gần bằng 1 — ví dụ: 0,9, 0,99 hoặc 0,999 (bạn muốn có nhiều 9 hơn cho các tập dữ liệu lớn hơn và các lô nhỏ nhỏ hơn).

Một siêu thông số quan trọng khác là trục: nó xác định trục nào nên được chuẩn hóa. Nó mặc định là –1, có nghĩa là theo mặc định, nó sẽ chuẩn hóa trục cuối cùng (sử dụng phương tiện và độ lệch chuẩn được tính trên các trục khác). Ví dụ: khi lô đầu vào là 2D (tức là hình dạng lô là [kích thước lô, tính năng]), điều này có nghĩa là
rằng mỗi tính năng đầu vào sẽ được chuẩn hóa dựa trên giá trị trung bình và độ lệch chuẩn được tính trên tất cả các trường hợp trong lô. Ví dụ: lớp BN đầu tiên trong ví dụ mã trước đó sẽ chuẩn hóa độc lập (và thay đổi tỷ lệ và dịch chuyển) từng tính năng đầu vào của 784. Tuy nhiên, nếu chúng ta di chuyển lớp BN đầu tiên trước lớp Flatten, thì các lô đầu vào sẽ là 3D, với hình dạng [kích thước lô, chiều cao, chiều rộng], do đó lớp BN sẽ tính 28 phương tiện và 28 độ lệch chuẩn (một trên mỗi cột của pixel, được tính trên tất cả các trường hợp trong lô và tất cả các hàng trong cột), và nó sẽ chuẩn hóa tất cả các pixel trong một cột nhất định bằng cách sử dụng cùng một giá trị trung bình và độ lệch chuẩn. Cũng sẽ chỉ có 28 thông số thang đo và 28 thông số dịch chuyển. Thay vào đó, nếu bạn vẫn muốn xử lý từng pixel trong số 784 pixel một cách độc lập, thì bạn nên đặt axis = [1, 2].

Chuẩn hóa hàng loạt đã trở thành một trong những lớp được sử dụng nhiều nhất trong mạng nơ-ron sâu, đến mức nó thường bị bỏ qua trong sơ đồ, vì người ta cho rằng BN được thêm vào sau mỗi lớp. Tuy nhiên, một bài báo (*“Fixup Initialization: Residual Learning Without Normalization,”*) gần đây của Hongyi Zhang et al. cũng có thể thay đổi điều này: các tác giả cho thấy rằng bằng cách sử dụng kỹ thuật khởi tạo trọng số cập nhật cố định (fixup) mới, họ quản lý để đào tạo một mạng nơ-ron rất sâu (10.000 lớp!) mà không cần BN, đạt được hiệu suất hiện đại trên các nhiệm vụ phân loại ảnh phức tạp.

### Cắt Gradient
Một kỹ thuật phổ biến khác để giảm bớt vấn đề về gradient đang bùng nổ là chỉ cần cắt các gradient trong quá trình nhân giống ngược để chúng không bao giờ vượt quá ngưỡng nào đó. Đây được gọi là Gradient Clipping (*“On the difficulty of training recurrent neural networks”*). Kỹ thuật này thường được sử dụng nhất trong các mạng nơ-ron tuần hoàn, vì Chuẩn hóa hàng loạt rất khó sử dụng trong RNN.

Trong Keras, việc triển khai Gradient Clipping chỉ là vấn đề thiết lập đối số clipvalue hoặc clipnorm khi tạo trình tối ưu hóa. Ví dụ:

```
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="mse", optimizer=optimizer)
```

Nếu bạn quan sát thấy gradient bùng nổ trong quá trình đào tạo (bạn có thể theo dõi kích thước của gradient bằng TensorBoard), bạn có thể muốn thử cả cắt theo giá trị và cắt theo định mức, với ngưỡng khác nhau và xem tùy chọn nào hoạt động tốt nhất trên bộ xác thực . 

## 11.2 Tái sử dụng Pretrained Layers
Nói chung không phải là một ý kiến hay để đào tạo một DNN rất lớn từ đầu: thay vào đó, bạn nên luôn cố gắng tìm một mạng nơ-ron hiện có hoàn thành nhiệm vụ tương tự với mạng mà bạn đang cố gắng giải quyết (chúng ta sẽ thảo luận về cách tìm chúng trong Chương 14), sau đó chỉ cần sử dụng lại các lớp thấp hơn của mạng này: đây được gọi là học chuyển giao (**transfer learning**). Nó không chỉ tăng tốc đáng kể cho quá trình đào tạo mà còn yêu cầu ít dữ liệu đào tạo hơn nhiều.

Ví dụ: giả sử bạn có quyền truy cập vào một DNN đã được huấn luyện để phân loại ảnh thành 100 loại khác nhau, bao gồm động vật, thực vật, xe cộ và đồ vật hàng ngày. Bây giờ bạn muốn đào tạo một DNN để phân loại các loại phương tiện cụ thể. Các tác vụ này rất giống nhau, thậm chí có phần chồng chéo, vì vậy bạn nên cố gắng sử dụng lại các phần của mạng đầu tiên (xem Hình 11-4).

![](images/11.4.png)

*Hình 11.4 Tái sử dụng Pretrained Layers*

> Nếu ảnh đầu vào của tác vụ mới của bạn không có cùng kích thước với ảnh được sử dụng trong tác vụ ban đầu, bạn thường sẽ phải thêm bước xử lý trước để thay đổi kích thước chúng thành kích thước mong đợi của mô hình ban đầu. Nói một cách tổng quát hơn, việc học chuyển giao sẽ hoạt động tốt nhất khi các đầu vào có các tính năng cấp thấp tương tự.

Lớp đầu ra của mô hình ban đầu thường nên được thay thế vì nó rất có thể không hữu ích cho tác vụ mới và thậm chí nó có thể không có số lượng đầu ra phù hợp cho tác vụ mới. 

Trước tiên, hãy thử đóng băng tất cả các lớp được sử dụng lại (tức là làm cho trọng lượng của chúng không thể đào tạo được, vì vậy độ dốc gradient sẽ không sửa đổi chúng), sau đó đào tạo mô hình của bạn và xem nó hoạt động như thế nào. Sau đó, hãy thử mở một hoặc hai trong số các lớp ẩn trên cùng để cho phép nhân giống ngược điều chỉnh chúng và xem liệu hiệu suất có được cải thiện hay không. Bạn càng có nhiều dữ liệu đào tạo, bạn càng có thể giải phóng nhiều lớp hơn. Nó cũng hữu ích để giảm tỷ lệ học tập khi bạn giải phóng các lớp được sử dụng lại: điều này sẽ tránh làm hỏng trọng lượng đã được tinh chỉnh của chúng.

Nếu bạn vẫn không thể có được hiệu suất tốt và bạn có ít dữ liệu đào tạo, hãy thử bỏ các lớp ẩn trên cùng và đóng băng tất cả các lớp ẩn còn lại một lần nữa. Bạn có thể lặp lại cho đến khi tìm được số lớp phù hợp để sử dụng lại. Nếu bạn có nhiều dữ liệu đào tạo, bạn có thể thử thay thế các lớp ẩn trên cùng thay vì bỏ chúng và thậm chí thêm nhiều lớp ẩn hơn.

### Transfer Learning với Keras
Hãy xem một ví dụ. Giả sử tập dữ liệu MNIST thời trang chỉ chứa 8 lớp, ví dụ như tất cả các lớp ngoại trừ dép và áo sơ mi. Một người nào đó đã xây dựng và đào tạo mô hình Keras trên bộ đó và có hiệu suất khá tốt (độ chính xác> 90%). Hãy gọi đây là mô hình A. Bây giờ bạn muốn giải quyết một nhiệm vụ khác: bạn có hình ảnh về dép và áo sơ mi và bạn muốn đào tạo một bộ phân loại nhị phân (dương = áo sơ mi, âm = dép). Tuy nhiên, tập dữ liệu của bạn khá nhỏ, bạn chỉ có 200 hình ảnh được gắn nhãn. Khi bạn đào tạo một mô hình mới cho nhiệm vụ này (hãy gọi nó là mô hình B), với kiến trúc giống như mô hình A, nó hoạt động khá tốt (độ chính xác 97,2%), nhưng vì nó là một nhiệm vụ dễ dàng hơn nhiều (chỉ có 2 lớp), bạn đã hy vọng nhiều hơn. Trong khi uống cà phê buổi sáng, bạn nhận ra rằng nhiệm vụ của mình khá giống với nhiệm vụ A, vậy có lẽ việc học chuyển tiếp có thể giúp ích? Hãy cùng tìm hiểu!

Trước tiên, bạn cần tải mô hình A và tạo mô hình mới dựa trên các lớp của mô hình A. Hãy sử dụng lại tất cả các lớp ngoại trừ lớp đầu ra:

In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

tf.random.set_seed(42)
np.random.seed(42)

In [6]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

model_A.save("my_model_A.h5")

Epoch 1/20
1375/1375 [==============================] - 3s 3ms/step - loss: 0.5926 - accuracy: 0.8103 - val_loss: 0.3895 - val_accuracy: 0.8665
Epoch 2/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.3523 - accuracy: 0.8784 - val_loss: 0.3287 - val_accuracy: 0.8824
Epoch 3/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.3171 - accuracy: 0.8896 - val_loss: 0.3014 - val_accuracy: 0.8994
Epoch 4/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2974 - accuracy: 0.8976 - val_loss: 0.2894 - val_accuracy: 0.9021
Epoch 5/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2836 - accuracy: 0.9020 - val_loss: 0.2775 - val_accuracy: 0.9061
Epoch 6/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2730 - accuracy: 0.9059 - val_loss: 0.2735 - val_accuracy: 0.9063
Epoch 7/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2642 - accuracy: 0.9094 - val_loss: 0.2721 - val_accuracy:

Lưu ý rằng `model_A` và `model_B_on_A` hiện chia sẻ một số lớp. Khi bạn đào tạo `model_B_on_A`, nó cũng sẽ ảnh hưởng đến `model_A`. Nếu bạn muốn tránh điều đó, bạn cần sao chép `model_A` trước khi sử dụng lại các lớp của nó. Để thực hiện việc này, bạn phải sao chép kiến trúc của mô hình A, sau đó sao chép các trọng số của nó (vì `clone_model()` không sao chép các trọng số):

Bây giờ chúng ta chỉ có thể huấn luyện `model_B_on_A` cho nhiệm vụ B, nhưng vì lớp đầu ra mới được khởi tạo ngẫu nhiên, nó sẽ tạo ra các lỗi lớn, ít nhất là trong vài epoch đầu tiên, vì vậy sẽ có các gradient lỗi lớn có thể phá hỏng các trọng số được sử dụng lại. Để tránh điều này, một cách tiếp cận là đóng băng các lớp được sử dụng lại trong vài kỷ nguyên đầu tiên, cho lớp mới một thời gian để tìm hiểu trọng lượng hợp lý. Để thực hiện việc này, chỉ cần đặt thuộc tính `trainable` của mọi lớp thành `False` và biên dịch mô hình:

In [8]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 0s 33ms/step - loss: 0.2000 - accuracy: 0.9600 - val_loss: 0.2155 - val_accuracy: 0.9665
Epoch 2/4
7/7 [==============================] - 0s 10ms/step - loss: 0.1943 - accuracy: 0.9600 - val_loss: 0.2101 - val_accuracy: 0.9665
Epoch 3/4
7/7 [==============================] - 0s 10ms/step - loss: 0.1888 - accuracy: 0.9600 - val_loss: 0.2048 - val_accuracy: 0.9696
Epoch 4/4
7/7 [==============================] - 0s 10ms/step - loss: 0.1837 - accuracy: 0.9600 - val_loss: 0.2000 - val_accuracy: 0.9716


Tiếp theo, chúng ta có thể đào tạo mô hình trong một vài epoch, sau đó giải phóng các lớp được sử dụng lại (yêu cầu biên dịch lại mô hình) và tiếp tục đào tạo để tinh chỉnh các lớp được sử dụng lại cho nhiệm vụ B. Sau khi giải phóng các lớp được sử dụng lại, nó thường là tốt ý tưởng để giảm tốc độ học, một lần nữa để tránh làm hỏng các trọng lượng được sử dụng lại:

In [9]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/16
7/7 [==============================] - 0s 42ms/step - loss: 0.1708 - accuracy: 0.9600 - val_loss: 0.1752 - val_accuracy: 0.9757
Epoch 2/16
7/7 [==============================] - 0s 12ms/step - loss: 0.1475 - accuracy: 0.9700 - val_loss: 0.1567 - val_accuracy: 0.9807
Epoch 3/16
7/7 [==============================] - 0s 12ms/step - loss: 0.1300 - accuracy: 0.9800 - val_loss: 0.1418 - val_accuracy: 0.9838
Epoch 4/16
7/7 [==============================] - 0s 12ms/step - loss: 0.1162 - accuracy: 0.9800 - val_loss: 0.1302 - val_accuracy: 0.9838
Epoch 5/16
7/7 [==============================] - 0s 11ms/step - loss: 0.1052 - accuracy: 0.9850 - val_loss: 0.1206 - val_accuracy: 0.9848
Epoch 6/16
7/7 [==============================] - 0s 12ms/step - loss: 0.0959 - accuracy: 0.9900 - val_loss: 0.1123 - val_accuracy: 0.9858
Epoch 7/16
7/7 [==============================] - 0s 12ms/step - loss: 0.0880 - accuracy: 0.9900 - val_loss: 0.1055 - val_accuracy: 0.9888
Epoch 8/16
7/7 [===========

Vậy, phán quyết cuối cùng là gì? Độ chính xác trong thử nghiệm của mô hình này là 99,25%, có nghĩa là việc học chuyển đổi đã giảm tỷ lệ lỗi từ 2,8% xuống gần 0,7%! Đó là hệ số 4!

In [11]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.0653 - accuracy: 0.9935


[0.06525216996669769, 0.9934999942779541]

Bạn có bị thuyết phục? Vâng, bạn không nên như vậy: Tôi đã lừa dối! :) Tôi đã thử nhiều cấu hình cho đến khi tôi tìm thấy một cấu hình cho thấy sự cải thiện mạnh mẽ. Nếu bạn cố gắng thay đổi các lớp hoặc hạt giống ngẫu nhiên, bạn sẽ thấy rằng sự cải thiện thường giảm xuống, hoặc thậm chí biến mất hoặc đảo ngược. Những gì tôi đã làm được gọi là "tra tấn dữ liệu cho đến khi nó thú nhận".
Khi một bài báo trông quá tích cực, bạn nên nghi ngờ: có lẽ kỹ thuật mới hào nhoáng không giúp được gì nhiều (trên thực tế, nó thậm chí có thể làm giảm hiệu suất), nhưng các tác giả đã thử nhiều biến thể và chỉ báo cáo kết quả tốt nhất (có thể do để cắt đứt may rủi), mà không cần đề cập đến bao nhiêu thất bại họ gặp phải trên đường đi.
Hầu hết thời gian, điều này không độc hại chút nào, nhưng nó là một phần lý do tại sao rất nhiều kết quả trong Khoa học không bao giờ có thể được tái tạo.

Vậy tại sao tôi lại lừa dối? Hóa ra là học chuyển giao không hoạt động tốt với các mạng nhỏ dày đặc: nó hoạt động tốt nhất với các mạng nơ-ron phức hợp sâu, vì vậy chúng ta sẽ xem lại việc học chuyển giao trong Chương 14, sử dụng các kỹ thuật tương tự (và lần này sẽ không có gian lận, Tôi hứa!).

### Unsupervised Pretraining (Đào tạo trước không được giám sát)

Giả sử bạn muốn giải quyết một nhiệm vụ phức tạp mà bạn không có nhiều dữ liệu đào tạo được gắn nhãn, nhưng tiếc là bạn không thể tìm thấy một mô hình được đào tạo về một nhiệm vụ tương tự. Đừng mất tất cả hy vọng! Trước tiên, tất nhiên bạn nên cố gắng thu thập nhiều dữ liệu huấn luyện được gắn nhãn hơn, nhưng nếu việc này quá khó hoặc quá đắt, bạn vẫn có thể thực hiện đào tạo trước không được giám sát (xem Hình 11-5). Việc thu thập các ví dụ đào tạo không được gắn nhãn thường khá rẻ, nhưng khá tốn kém để gắn nhãn chúng. Nếu bạn có thể thu thập nhiều dữ liệu đào tạo không được gắn nhãn, bạn có thể cố gắng đào tạo từng lớp một, bắt đầu với lớp thấp nhất và sau đó đi lên, sử dụng thuật toán dò tính năng không được giám sát như Restricted Boltzmann Machines (RBMs) hoặc mã tự động (autoencoders). Mỗi lớp được đào tạo dựa trên đầu ra của các lớp đã được đào tạo trước đó (tất cả các lớp ngoại trừ lớp đang được đào tạo đều được đóng băng). Khi tất cả các lớp đã được đào tạo theo cách này, bạn có thể thêm lớp đầu ra cho nhiệm vụ của mình và tinh chỉnh mạng cuối cùng bằng cách sử dụng học có giám sát (tức là với các ví dụ đào tạo được gắn nhãn). Tại thời điểm này, bạn có thể giải phóng tất cả các lớp đã được xử lý trước hoặc chỉ một số lớp phía trên.

![](images/11.5.png)

Đây là một quá trình khá dài và tẻ nhạt, nhưng nó thường hoạt động tốt; Trên thực tế, chính kỹ thuật này mà Geoffrey Hinton và nhóm của ông đã sử dụng vào năm 2006 và đã dẫn đến sự hồi sinh của mạng nơ-ron và sự thành công của Học sâu. Cho đến năm 2010, đào tạo trước không giám sát (thường sử dụng RBM) là tiêu chuẩn cho các mạng lưới sâu và chỉ sau khi vấn đề độ dốc biến mất được giảm bớt, việc đào tạo DNN hoàn toàn bằng cách sử dụng học có giám sát trở nên phổ biến hơn nhiều. Tuy nhiên, đào tạo trước không được giám sát (ngày nay thường sử dụng mã tự động thay vì RBM) vẫn là một lựa chọn tốt khi bạn có một nhiệm vụ phức tạp cần giải quyết, không có mô hình tương tự nào bạn có thể sử dụng lại và ít dữ liệu đào tạo được gắn nhãn nhưng nhiều dữ liệu đào tạo chưa được gắn nhãn.

### Pretraining on an Auxiliary Task
Nếu bạn không có nhiều dữ liệu đào tạo được gắn nhãn, một tùy chọn cuối cùng là đào tạo mạng nơ-ron đầu tiên trên một tác vụ phụ trợ mà bạn có thể dễ dàng lấy hoặc tạo dữ liệu đào tạo được gắn nhãn, sau đó sử dụng lại các lớp dưới của mạng đó cho tác vụ thực tế của bạn. Các lớp thấp hơn của mạng nơ-ron đầu tiên sẽ tìm hiểu các trình phát hiện tính năng có thể sẽ được sử dụng lại bởi mạng nơ-ron thứ hai.

Ví dụ: nếu bạn muốn xây dựng một hệ thống nhận dạng khuôn mặt, bạn có thể chỉ có một vài hình ảnh của mỗi cá nhân — rõ ràng là không đủ để đào tạo một người phân loại tốt. Thu thập hàng trăm bức ảnh của mỗi người sẽ không thực tế. Tuy nhiên, bạn có thể thu thập rất nhiều ảnh của những người ngẫu nhiên trên web và đào tạo một mạng thần kinh đầu tiên để phát hiện xem hai bức ảnh khác nhau có cùng một người hay không. Một mạng như vậy sẽ học được các bộ phát hiện tính năng tốt cho khuôn mặt, vì vậy việc sử dụng lại các lớp thấp hơn của nó sẽ cho phép bạn đào tạo một bộ phân loại khuôn mặt tốt bằng cách sử dụng ít dữ liệu đào tạo.

Đối với các ứng dụng xử lý ngôn ngữ tự nhiên (NLP), bạn có thể dễ dàng tải xuống hàng triệu tài liệu văn bản và tự động tạo dữ liệu được gắn nhãn từ đó. Ví dụ: bạn có thể che dấu ngẫu nhiên một số từ và đào tạo một mô hình để dự đoán những từ còn thiếu là gì (ví dụ: nó sẽ dự đoán rằng từ bị thiếu trong câu `“What ___you said?”` Có thể là `“are”` hoặc `“were”`) . Nếu bạn có thể đào tạo một mô hình để đạt được hiệu suất tốt trong nhiệm vụ này, thì nó sẽ biết khá nhiều về ngôn ngữ và bạn chắc chắn có thể sử dụng lại nó cho nhiệm vụ thực tế của mình và tinh chỉnh nó trên dữ liệu được gắn nhãn của bạn.

## 11.3 Trình tối ưu hóa nhanh hơn

Việc đào tạo một mạng lưới thần kinh sâu rất lớn có thể rất chậm. Cho đến nay, chúng tôi đã thấy bốn cách để tăng tốc quá trình đào tạo (và đạt được giải pháp tốt hơn): áp dụng chiến lược khởi tạo tốt cho trọng số kết nối, sử dụng hàm kích hoạt tốt, sử dụng Batch Normalization và sử dụng lại các phần của mạng được đào tạo trước (có thể được xây dựng trên một nhiệm vụ bổ trợ hoặc sử dụng phương pháp học tập không có giám sát). Một tốc độ tăng đáng kể khác đến từ việc sử dụng trình tối ưu hóa nhanh hơn trình tối ưu hóa Gradient Descent thông thường. Trong phần này, chúng tôi sẽ trình bày những cái phổ biến nhất: Tối ưu hóa Momentum, Nesterov Accelerated Gradient, AdaGrad, RMSProp, và cuối cùng là tối ưu hóa Adam và Nadam.

### Tối ưu hóa động lượng (Momentum )

Hãy tưởng tượng một quả bóng bowling lăn xuống dốc nhẹ trên một bề mặt nhẵn: nó sẽ khởi động chậm, nhưng nó sẽ nhanh chóng lấy đà cho đến khi đạt đến vận tốc cuối cùng (nếu có một số ma sát hoặc lực cản của không khí). Đây là ý tưởng rất đơn giản đằng sau việc tối ưu hóa Momentum, được đề xuất bởi Boris Polyak vào năm 1964 (* "Some methods of speeding up the convergence of iteration methods"*). Ngược lại, Gradient Descent thông thường sẽ chỉ đơn giản là thực hiện các bước nhỏ đều đặn xuống dốc, vì vậy sẽ mất nhiều thời gian hơn để đạt đến đáy.

Nhớ lại rằng Gradient Descent chỉ cập nhật trọng số $θ$ bằng cách trừ trực tiếp gradient của hàm chi phí $J(θ)$ liên quan đến trọng số ($∇_θJ(θ)$) nhân với tốc độ học $η$. Phương trình là: $θ ← θ - η∇_θJ(θ)$. Nó không quan tâm đến những gì các gradient trước đó là. Nếu gradient cục bộ nhỏ, nó sẽ đi rất chậm.

Tối ưu hóa động lượng quan tâm rất nhiều đến các gradient trước đó là gì: tại mỗi lần lặp, nó trừ đi gradient cục bộ từ vectơ động lượng m (nhân với tốc độ học η) và nó cập nhật trọng số bằng cách chỉ cần thêm vectơ động lượng này (xem Công thức 11 -4). Nói cách khác, gradient được sử dụng cho gia tốc, không phải cho tốc độ. Để mô phỏng một số loại cơ chế ma sát và ngăn động lượng phát triển quá lớn, thuật toán giới thiệu một siêu thông số $β$ mới, được gọi đơn giản là động lượng, phải được đặt giữa 0 (ma sát cao) và 1 (không ma sát). Giá trị động lượng điển hình là 0,9.

*Phương trình 11-4. Thuật toán động lượng*

1. $m ← βm − η∇_θJ(θ)$
2. $θ ← θ + m$

### Nesterov Accelerated Gradient
Một biến thể nhỏ của tối ưu hóa Momentum, được đề xuất bởi Yurii Nesterov vào năm 1983 (*"A Method for Unconstrained Convex Minimization Problem with the Rate of Convergence $O(1/k^2)$"*), hầu như luôn nhanh hơn so với tối ưu hóa Momentum vani. Ý tưởng của tối ưu hóa Nesterov Momentum, hoặc Nesterov Accelerated Gradient (NAG), là để đo độ dốc của hàm chi phí không phải ở vị trí cục bộ mà hơi về phía trước theo hướng của động lượng (xem Công thức 11-5). Sự khác biệt duy nhất so với tối ưu hóa Momentum vani là gradient được đo ở $θ + βm$ chứ không phải ở $θ$.

*Phương trình 11-5. Thuật toán Nesterov Accelerated Gradient*

1. $m ← βm − η∇_θJ(θ + βm)$
2. $θ ← θ + m$

Điều chỉnh nhỏ này hoạt động bởi vì nói chung vectơ động lượng sẽ hướng về đúng hướng (tức là về hướng tối ưu), vì vậy sẽ chính xác hơn một chút nếu sử dụng gradient được đo xa hơn một chút theo hướng đó thay vì sử dụng gradient ở vị trí ban đầu, như bạn có thể thấy trong Hình 11-6 (trong đó $∇_1$ đại diện cho gradient của hàm chi phí được đo tại điểm bắt đầu $θ$ và $∇_2$ đại diện cho gradient tại điểm nằm ở $θ + βm$). Như bạn có thể thấy, bản cập nhật Nesterov kết thúc gần với mức tối ưu hơn một chút. Sau một thời gian, những cải tiến nhỏ này cộng lại và NAG kết thúc nhanh hơn đáng kể so với tối ưu hóa Momentum thông thường. Hơn nữa, hãy lưu ý rằng khi xung lượng đẩy quả nặng qua một thung lũng, $∇_1$ tiếp tục đẩy xa hơn qua thung lũng, trong khi $∇_2$ đẩy trở lại đáy thung lũng. Điều này giúp giảm dao động và do đó hội tụ nhanh hơn.

![](images/11.6.png)

*Hình 11.6. Regular với Nesterov Momentum optimization *

NAG hầu như sẽ luôn tăng tốc độ đào tạo so với việc tối ưu hóa Momentum thông thường. Để sử dụng nó, chỉ cần đặt `nesterov = True` khi tạo trình tối ưu hóa SGD:
```
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)
```

### AdaGrad
Hãy xem xét vấn đề hình bát kéo dài một lần nữa: Gradient Descent bắt đầu bằng cách nhanh chóng đi xuống con dốc lớn nhất, sau đó từ từ đi xuống đáy của thung lũng. Sẽ thật tuyệt nếu thuật toán có thể phát hiện điều này sớm và sửa hướng của nó để hướng nhiều hơn một chút đến mức tối ưu toàn cầu.

Thuật toán AdaGrad (*"Adaptive Subgradient Methods for Online Learning and Stochastic Optimization"*) đạt được điều này bằng cách thu nhỏ vectơ gradient dọc theo các kích thước dốc nhất (xem Công thức 11-6):

*Phương trình 11-6. AdaGrad algorithm*

1. $s ← s + ∇_θJ(θ) ⊗ ∇_θJ(θ)$
2. $θ ← θ − η ∇_θJ(θ) ⊘ \sqrt{s + ϵ} $

Bước đầu tiên tích lũy bình phương của các gradient vào vectơ s (nhớ lại rằng ký hiệu ⊗ đại diện cho phép nhân element-wise). Dạng vectơ hóa này tương đương với tính toán $s_i ← s_i + (∂J(θ) / ∂θ_i)^2$ cho mỗi phần tử $s_i$ của vectơ $s$; nói cách khác, mỗi $s_i$ tích lũy các bình phương của đạo hàm riêng của hàm chi phí liên quan đến tham số $θ_i$. Nếu hàm chi phí dốc dọc theo thứ nguyên thứ i, thì $s_i$ sẽ lớn hơn và lớn hơn ở mỗi lần lặp.

Bước thứ hai gần giống với Gradient Descent, nhưng có một điểm khác biệt lớn: vector gradient được thu nhỏ theo hệ số $\sqrt{s + ϵ}$ (biểu tượng $⊘$ đại diện cho phép chia theo phần tử và $ϵ$ là một thuật ngữ làm mịn để tránh chia cho 0, thường được đặt thành $10^{–10}$). Dạng vectơ hóa này tương đương với tính toán $θ_i ←
 θ_i - η∂J(θ)/∂θ_i/\sqrt{s + ϵ} $cho tất cả các tham số θi (đồng thời).

Nói tóm lại, thuật toán này làm giảm tốc độ học tập, nhưng nó làm nhanh hơn đối với các kích thước dốc hơn so với các kích thước có độ dốc nhẹ hơn. Đây được gọi là tỷ lệ học tập thích ứng (*adaptive learning rate*). Nó giúp hướng các cập nhật kết quả trực tiếp hơn tới mức tối ưu toàn cầu (xem Hình 11-7). Một lợi ích bổ sung là nó yêu cầu điều chỉnh ít hơn nhiều của siêu thông số tốc độ học $η$.

![](images/11.7.png)

*Hình 11.7 AdaGrad với Gradient Descent*

AdaGrad thường hoạt động tốt đối với các bài toán bậc hai đơn giản, nhưng không may là nó thường dừng lại quá sớm khi đào tạo mạng nơ-ron. Tỷ lệ học tập được thu nhỏ đến mức thuật toán kết thúc hoàn toàn dừng lại trước khi đạt đến mức tối ưu toàn cầu. Vì vậy, mặc dù Keras có trình tối ưu hóa Adagrad, bạn không nên sử dụng nó để đào tạo các mạng thần kinh sâu (tuy nhiên, nó có thể hiệu quả đối với các tác vụ đơn giản hơn như Hồi quy tuyến tính). Tuy nhiên, hiểu Adagrad sẽ hữu ích để nắm được các công cụ tối ưu hóa tốc độ học tập thích ứng khác.

### RMSProp
Mặc dù AdaGrad làm chậm quá nhanh và kết thúc là không bao giờ hội tụ đến mức tối ưu toàn cầu, thuật toán RMSProp khắc phục điều này bằng cách chỉ tích lũy các gradient từ các lần lặp gần đây nhất (trái ngược với tất cả các gradient kể từ khi bắt đầu đào tạo). Nó làm như vậy bằng cách sử dụng phân rã theo cấp số nhân trong bước đầu tiên (xem Công thức 11-7).

*Phương trình 11-7. Thuật toán RMSProp*

1. $s ← βs + (1 − β) ∇_θJ(θ) ⊗ ∇_θJ(θ)$
2. $θ ← θ − η ∇_θJ (θ) ⊘\sqrt{s + ϵ}$

Tốc độ phân rã $β$ thường được đặt thành 0,9. Đúng, nó lại là một siêu thông số mới, nhưng giá trị mặc định này thường hoạt động tốt, vì vậy bạn có thể không cần điều chỉnh nó.

Như bạn có thể mong đợi, Keras có trình tối ưu hóa `RMSProp`:

```
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)
```

Ngoại trừ những vấn đề rất đơn giản, trình tối ưu hóa này hầu như luôn hoạt động tốt hơn nhiều so với AdaGrad. Trên thực tế, nó là thuật toán tối ưu hóa ưa thích của nhiều nhà nghiên cứu cho đến khi tối ưu hóa Adam xuất hiện.

### Adam and Nadam Optimization
Adam (adaptive moment estimation), kết hợp các ý tưởng của tối ưu hóa Momentum và RMSProp: giống như tối ưu hóa Momentum, nó theo dõi mức trung bình giảm dần theo cấp số nhân của các gradient trong quá khứ và giống như RMSProp, nó theo dõi mức trung bình giảm dần theo cấp số nhân của các gradient bình phương trong quá khứ (xem Công thức 11-8).

*Phương trình 11-8. Thuật toán Adam*

1. $m ← β_1m − (1 − β_1) ∇_θJ(θ)$
2. $s ← β_2s + (1 − β_2) ∇_θJ (θ) ⊗ ∇_θJ(θ)$
3. $\hat{m} ← \frac{m }{1 − β_1^t}$
4. $\hat{s} ← \frac{s} {1 − β_2^t}$
5. $θ ← θ + η \hat{m} ⊘\sqrt{\hat{s} + ϵ}$
- $t$ đại diện cho số lần lặp lại (iteration) (bắt đầu từ 1)

Nếu bạn chỉ xem xét các bước 1, 2 và 5, bạn sẽ nhận thấy sự tương đồng gần của Adam với cả tối ưu hóa Momentum và RMSProp. Sự khác biệt duy nhất là bước 1 tính toán trung bình giảm dần theo cấp số nhân chứ không phải là tổng phân rã theo cấp số nhân, nhưng chúng thực sự tương đương ngoại trừ một hệ số không đổi (trung bình phân rã chỉ bằng $1 - β_1$ lần tổng phân rã). Bước 3 và 4 là một chi tiết kỹ thuật: vì $m$ và $s$ được khởi tạo bằng 0, chúng sẽ thiên về 0 khi bắt đầu đào tạo, vì vậy hai bước này sẽ giúp tăng cường $m$ và $s$ khi bắt đầu đào tạo.

Siêu tham số phân rã động lượng $β_1$ thường được khởi tạo bằng $0,9$, trong khi siêu thông số phân rã theo tỷ lệ $β_2$ thường được khởi tạo thành $0,999$. Như trước đó, thuật ngữ làm mịn $ϵ$ thường được khởi tạo thành một số nhỏ chẳng hạn như $10^{–7}$. Đây là các giá trị mặc định cho lớp Adam (nói chính xác, epsilon mặc định là Không có, điều này yêu cầu Keras sử dụng `keras.backend.epsilon()`, mặc định là $10^{–7}$; bạn có thể thay đổi nó bằng cách sử dụng `keras.backend.set_epsilon( )`)

```
optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
```

Vì Adam là một thuật toán tốc độ học tập thích ứng (như AdaGrad và RMSProp), nó yêu cầu ít điều chỉnh hơn siêu thông số learning rate $η$. Bạn thường có thể sử dụng giá trị mặc định $η = 0,001$, làm cho Adam thậm chí còn dễ sử dụng hơn Gradient Descent.

Tất cả các kỹ thuật tối ưu hóa được thảo luận cho đến nay chỉ dựa trên các đạo hàm riêng bậc nhất (Jacobians). Các tài liệu về tối ưu hóa chứa các thuật toán tuyệt vời dựa trên các đạo hàm riêng bậc hai (các Hessian, là các đạo hàm riêng của các Jacobian). Thật không may, các thuật toán này rất khó áp dụng cho mạng nơ-ron sâu vì có $n^2$ Hessian trên mỗi đầu ra (trong đó $n$ là số tham số), trái ngược với chỉ $n$ Jacobian trên mỗi đầu ra. Vì DNN thường có hàng chục nghìn tham số, các thuật toán tối ưu hóa bậc hai thường thậm chí không phù hợp với bộ nhớ và ngay cả khi chúng làm như vậy, việc tính toán Hessians cũng quá chậm.

### Learning Rate Scheduling

Tìm một tỷ lệ học tập tốt có thể khó khăn. Nếu bạn đặt nó quá cao, quá trình đào tạo có thể thực sự khác nhau (như chúng ta đã thảo luận trong Chương 4). Nếu bạn đặt nó quá thấp, quá trình đào tạo cuối cùng sẽ hội tụ đến mức tối ưu, nhưng sẽ mất một thời gian rất dài. Nếu bạn đặt nó hơi cao, lúc đầu nó sẽ đạt được tiến bộ rất nhanh, nhưng cuối cùng nó sẽ nhảy múa xung quanh mức tối ưu, không bao giờ thực sự lắng xuống. Nếu bạn có một ngân sách máy tính hạn chế, bạn có thể phải gián đoạn việc đào tạo trước khi nó hội tụ đúng cách, mang lại một giải pháp không tối ưu (xem Hình 11-8).

![](images/11.8.png)

Như chúng ta đã thảo luận trong Chương 10, một cách tiếp cận là bắt đầu với tốc độ học tập lớn và chia nó cho 3 cho đến khi thuật toán huấn luyện ngừng phân kỳ. Bạn sẽ không còn quá xa so với tỷ lệ học tối ưu, sẽ tiếp thu nhanh và hội tụ cách giải tốt.
Tuy nhiên, bạn có thể làm tốt hơn tốc độ học tập không đổi: nếu bạn bắt đầu với tốc độ học tập cao và sau đó giảm xuống khi nó ngừng tiến bộ nhanh, bạn có thể đạt được giải pháp tốt nhanh hơn so với tốc độ học tập không đổi tối ưu. Có nhiều chiến lược khác nhau để giảm tỷ lệ học tập trong quá trình đào tạo. Các chiến lược này được gọi là learning schedules (chúng tôi đã giới thiệu ngắn gọn khái niệm này trong Chương 4), trong đó phổ biến nhất là:

- **Power scheduling**: Đặt tốc độ học thành một hàm của số lặp $t$: $η (t) = η_0 / (1 + t / k)^c$. Tốc độ học ban đầu η0, công suất $c$ (thường được đặt thành 1) và các bước $s$ là siêu tham số. Tốc độ học tập giảm xuống ở mỗi bước, và sau một số bước, tốc độ giảm xuống $η_0 / 2$. Sau một số bước nữa, tốc độ giảm xuống $η_0 / 3$. Sau đó giảm xuống $η_0 / 4$, rồi $η_0 / 5$, v.v. Như bạn có thể thấy, lịch trình này đầu tiên giảm nhanh, sau đó ngày càng chậm hơn. Tất nhiên, điều này yêu cầu điều chỉnh $η_0$, $s$ (và có thể là $c$).
- **Exponential scheduling** (theo cấp số nhân): Đặt tốc độ học là: $η (t) = η_0 0,1^{t / s}$. Tốc độ học tập sẽ giảm dần theo hệ số 10 sau mỗi $s$ bước. Trong khi lập lịch nguồn làm giảm tốc độ học tập ngày càng chậm hơn, lập lịch theo cấp số nhân tiếp tục cắt giảm nó đi 10 bước mỗi s.
- **Piecewise constant scheduling** (Lập kế hoạch liên tục từng mảnh): Sử dụng tốc độ học không đổi cho một số epoch (ví dụ: $η_0 = 0,1$ trong 5 epoch), sau đó tốc độ học tập nhỏ hơn cho một số kỷ nguyên khác (ví dụ: $η_1 = 0,001$ cho 50 epoch), v.v. Mặc dù giải pháp này có thể hoạt động rất tốt, nhưng nó đòi hỏi bạn phải mày mò để tìm ra chuỗi tỷ lệ học tập phù hợp và thời gian sử dụng từng tỷ lệ đó.
- **Performance scheduling**: Đo lỗi xác nhận sau mỗi $N$ bước (giống như khi dừng sớm) và giảm tốc độ học theo hệ số $λ$ khi lỗi ngừng giảm.

Một bài báo *An Empirical Study of Learning Rates in Deep Neural Networks for Speech Recognition* năm 2013 của Andrew Senior et al. đã so sánh hiệu suất của một số lịch trình học tập phổ biến nhất khi đào tạo mạng thần kinh sâu để nhận dạng giọng nói bằng cách sử dụng tối ưu hóa Momentum. Các tác giả kết luận rằng, trong cài đặt này, performance scheduling và exponential scheduling đều hoạt động tốt. Họ thích exponential scheduling vì nó dễ điều chỉnh và nó hội tụ nhanh hơn một chút đến giải pháp tối ưu (họ cũng đề cập rằng nó dễ thực hiện hơn performance scheduling, nhưng trong Keras, cả hai tùy chọn đều dễ dàng).

Thực hiện power scheduling trong Keras là tùy chọn dễ dàng nhất: chỉ cần đặt siêu thông số `decay` khi tạo trình tối ưu hóa. `decay` là nghịch đảo của $s$ (số bước cần thiết để chia tốc độ học thêm một đơn vị) và Keras giả định rằng $c$ bằng 1. Ví dụ:

```
optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-4)
```

Exponential scheduling và piecewise scheduling cũng khá đơn giản. Trước tiên, bạn cần xác định một hàm lấy epoch hiện tại và trả về tốc độ học. Ví dụ: hãy triển khai exponential scheduling:
```
def exponential_decay_fn(epoch):
    return 0.01 * 0.1**(epoch / 20)
```

Nếu bạn không muốn mã hóa cứng $η_0$ và $s$, bạn có thể tạo một hàm trả về một hàm đã định cấu hình:
```
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1**(epoch / s)
    return exponential_decay_fn
    
exponential_decay_fn = exponential_decay(lr0=0.01, s=20)
```

Tiếp theo, chỉ cần tạo một lệnh gọi lại `LearningRateScheduler`, cung cấp cho nó hàm lịch biểu và chuyển lệnh gọi lại này đến phương thức `fit ()`:

```
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
history = model.fit(X_train_scaled, y_train, [...], callbacks=[lr_scheduler])
```
`LearningRateScheduler` sẽ cập nhật thuộc tính `learning_rate` của trình tối ưu hóa vào đầu mỗi epoch. Cập nhật tốc độ học chỉ một lần cho mỗi epoch thường là đủ, nhưng nếu bạn muốn nó được cập nhật thường xuyên hơn, chẳng hạn như ở mỗi bước, bạn cần phải viết lệnh gọi lại của riêng mình . Điều này có thể có ý nghĩa nếu có nhiều bước cho mỗi epoch.

Khi bạn lưu một mô hình, trình tối ưu hóa và tốc độ học tập của nó sẽ được lưu cùng với nó. Điều này có nghĩa là với chức năng lịch trình mới này, bạn chỉ có thể tải một mô hình đã được đào tạo và tiếp tục đào tạo từ những nơi nó đã dừng lại, không có vấn đề gì. Tuy nhiên, mọi thứ không đơn giản như vậy nếu hàm lịch biểu của bạn sử dụng đối số epoch: thực sự, epoch không được lưu và nó được đặt lại về 0 mỗi khi bạn gọi phương thức `fit()`. Điều này có thể dẫn đến tỷ lệ học tập rất lớn khi bạn tiếp tục đào tạo một mô hình mà nó đã dừng lại, điều này có thể làm hỏng trọng lượng của mô hình của bạn. Một giải pháp là đặt đối số `initial_epoc`h của phương thức `fit()` theo cách thủ công để epoch bắt đầu ở giá trị phù hợp.

Đối với  piecewise constant scheduling, bạn có thể sử dụng một hàm lịch biểu như sau (như trước đó, bạn có thể xác định một hàm tổng quát hơn nếu muốn, hãy xem sổ tay để biết ví dụ), sau đó tạo lệnh gọi lại `LearningRateScheduler` với hàm này và chuyển nó đến phương thức `fit()`, giống như chúng tôi đã làm đối với exponential scheduling:

```
def piecewise_constant_fn(epoch):
    if epoch < 5:
        return 0.01
    elif epoch < 15:
        return 0.005
    else:
        return 0.001
```
Để performance scheduling, chỉ cần sử dụng lệnh gọi lại `ReduceLROnPlateau`. Ví dụ: nếu bạn chuyển lệnh gọi lại sau cho phương thức `fit()`, nó sẽ nhân tỷ lệ học tập với 0,5 bất cứ khi nào mất xác thực tốt nhất không cải thiện trong 5 kỷ nguyên liên tiếp (có sẵn các tùy chọn khác, vui lòng kiểm tra tài liệu để biết thêm chi tiết) :

```
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
```

Cuối cùng, `tf.keras` cung cấp một cách thay thế để triển khai lập lịch tốc độ học tập: chỉ cần xác định tốc độ học tập bằng cách sử dụng một trong các lịch trình có sẵn trong `keras.optimizers.schedules`, sau đó chuyển tốc độ học tập này cho bất kỳ trình tối ưu hóa nào. Cách tiếp cận này cập nhật tỷ lệ học tập ở mỗi bước thay vì ở mỗi epoch. Ví dụ: đây là cách triển khai cùng một lịch biểu theo cấp số nhân như trước đó:

```
s = 20 * len(X_train) // 32 # number of steps in 20 epochs (batch size = 32)
learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
optimizer = keras.optimizers.SGD(learning_rate)
```
Điều này rất hay và đơn giản, cộng với khi bạn lưu mô hình, tốc độ học tập và lịch trình của nó (bao gồm cả trạng thái của nó) cũng sẽ được lưu. Tuy nhiên, cách tiếp cận này không phải là một phần của API Keras, nó dành riêng cho `tf.keras`.

Tóm lại, phân rã theo cấp số nhân (exponential decay) hoặc lập lịch hiệu suất (performance scheduling) có thể tăng tốc độ hội tụ đáng kể, vì vậy hãy thử chúng!

## 11.4 Tránh Overfittingc thông qua Regularization
Với hàng ngàn thông số bạn có thể phù hợp với toàn bộ vườn thú. Mạng nơ-ron sâu thường có hàng chục nghìn tham số, đôi khi thậm chí hàng triệu. Với rất nhiều tham số, mạng có một lượng tự do đáng kinh ngạc và có thể phù hợp với nhiều loại tập dữ liệu phức tạp. Nhưng sự linh hoạt tuyệt vời này cũng có nghĩa là nó dễ bị trang bị quá mức cho bộ đào tạo. Chúng tôi cần chính quy hóa (regularization).

Chúng tôi đã thực hiện một trong những kỹ thuật chính quy tốt nhất trong Chương 10: dừng sớm. Hơn nữa, mặc dù Batch Normalization được thiết kế để giải quyết các vấn đề về tiêu biến/bùng nổ gradien, nó cũng hoạt động giống như một bộ điều chỉnh khá tốt. Trong phần này, chúng tôi sẽ trình bày các kỹ thuật chính quy phổ biến khác cho mạng nơ-ron: chính quy $ℓ_1$ và $ℓ_2$, dropout và max-norm regularization.

### $ℓ_1$ và $ℓ_2$ Regularization
Giống như bạn đã làm trong Chương 4 đối với các mô hình tuyến tính đơn giản, bạn có thể sử dụng chính quy $ℓ_1$ và $ℓ_2$ để hạn chế trọng số kết nối của mạng nơ-ron (nhưng thường không phải là độ lệch (biases) của nó). Dưới đây là cách áp dụng chính quy $ℓ_2$ cho trọng số kết nối của lớp Keras, sử dụng hệ số chính quy 0,01:

```
layer = keras.layers.Dense(100, activation="elu",
                            kernel_initializer="he_normal",
                            kernel_regularizer=keras.regularizers.l2(0.01))
```
Hàm `l2()` trả về một bộ điều chỉnh sẽ được gọi để tính toán sự mất mát chính quy, ở mỗi bước trong quá trình huấn luyện. Tổn thất chính quy này sau đó được cộng vào tổn thất cuối cùng. Như bạn có thể mong đợi, bạn chỉ có thể sử dụng `keras.regularizers.l1()` nếu bạn muốn chính quy $ℓ_1$ và nếu bạn muốn chính quy hóa $ℓ_1$ và $ℓ_2$, hãy sử dụng `keras.regularizers.l1_l2()` (chỉ định cả hai yếu tố chính quy).

Vì bạn thường muốn áp dụng cùng một bộ điều chỉnh cho tất cả các lớp trong mạng của mình, cũng như cùng một chức năng kích hoạt và cùng một chiến lược khởi tạo trong tất cả các lớp ẩn, bạn có thể thấy mình lặp đi lặp lại các đối số giống nhau. Điều này làm cho nó trở nên xấu xí và dễ bị lỗi. Để tránh điều này, bạn có thể thử cấu trúc lại mã của mình để sử dụng các vòng lặp. Một tùy chọn khác là sử dụng hàm `functools.partial()` của Python: nó cho phép bạn tạo một trình bao bọc mỏng cho bất kỳ hàm có thể gọi nào, với một số giá trị đối số mặc định. Ví dụ:
```
from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                            activation="elu",
                            kernel_initializer="he_normal",
                            kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax",
                        kernel_initializer="glorot_uniform")
])
```

### Dropout
Bỏ học (Dropout) là một trong những kỹ thuật điều chỉnh phổ biến nhất cho mạng nơ-ron sâu. Nó đã được đề xuất bởi Geoffrey Hinton vào năm 2012 và được trình bày chi tiết hơn trong một bài báo (*"Dropout: A Simple Way to Prevent Neural Networks from Overfittin"*)  bởi Nitish Srivastava và cộng sự, và nó đã được chứng minh là rất thành công: ngay cả các mạng nơ-ron hiện đại cũng tăng độ chính xác lên 1–2% chỉ bằng cách thêm bỏ . Điều này nghe có vẻ không nhiều, nhưng khi một mô hình đã có độ chính xác 95%, việc tăng độ chính xác lên 2% đồng nghĩa với việc giảm tỷ lệ lỗi gần 40% (giảm từ 5% lỗi xuống khoảng 3%).

Đây là một thuật toán khá đơn giản: ở mỗi bước huấn luyện, mọi nơ-ron (bao gồm các nơ-ron đầu vào, nhưng luôn loại trừ các nơ-ron đầu ra) đều có xác suất $p$ tạm thời bị “loại bỏ”, nghĩa là nó sẽ hoàn toàn bị bỏ qua trong bước đào tạo này, nhưng nó có thể hoạt động trong bước tiếp theo (xem Hình 11-9). Siêu tham số $p$ được gọi là tỷ lệ bỏ học, và nó thường được đặt thành 50%. Sau khi đào tạo, các tế bào thần kinh không bị giảm nữa. Và đó là tất cả (ngoại trừ một chi tiết kỹ thuật mà chúng ta sẽ thảo luận trong giây lát).

![](images/11.9.png)

*Hình 11.9  Dropout regularization*

Lúc đầu, khá ngạc nhiên là kỹ thuật khá tàn bạo này lại có tác dụng. Liệu một công ty có hoạt động tốt hơn nếu nhân viên của họ được yêu cầu tung một đồng xu mỗi sáng để quyết định có đi làm hay không? Chà, ai mà biết được; có lẽ nó sẽ! Công ty rõ ràng sẽ buộc phải điều chỉnh tổ chức của mình; nó không thể dựa vào bất kỳ người nào để điền vào máy pha cà phê hoặc thực hiện bất kỳ nhiệm vụ quan trọng nào khác, vì vậy chuyên môn này sẽ phải được phổ biến cho nhiều người. Nhân viên sẽ phải học cách hợp tác với nhiều đồng nghiệp của họ, không chỉ một số ít trong số họ. Công ty sẽ trở nên linh hoạt hơn nhiều. Nếu một người bỏ việc, điều đó sẽ không tạo ra nhiều khác biệt. Không rõ liệu ý tưởng này có thực sự hiệu quả với các công ty hay không, nhưng chắc chắn nó có hiệu quả với các mạng thần kinh. Các tế bào thần kinh được huấn luyện khi bỏ học không thể cùng thích ứng với các tế bào thần kinh lân cận của chúng; chúng phải trở nên hữu ích nhất có thể. Họ cũng không thể phụ thuộc quá nhiều vào chỉ một vài nơ-ron đầu vào; họ phải chú ý đến từng tế bào thần kinh đầu vào của mình. Cuối cùng chúng ít nhạy cảm hơn với những thay đổi nhỏ trong đầu vào. Cuối cùng, bạn sẽ có được một mạng lưới mạnh mẽ hơn, tổng quát hóa tốt hơn.

Một cách khác để hiểu sức mạnh của việc bỏ học là nhận ra rằng một mạng nơ-ron duy nhất được tạo ra ở mỗi bước đào tạo. Vì mỗi nơ-ron có thể có hoặc không có, nên có tổng cộng 2N mạng khả thi (trong đó N là tổng số nơ-ron có thể loại bỏ). Đây là một con số khổng lồ đến nỗi hầu như không thể có cùng một mạng nơ-ron được lấy mẫu hai lần. Khi bạn đã chạy 10.000 bước đào tạo, về cơ bản bạn đã đào tạo 10.000 mạng nơ-ron khác nhau (mỗi mạng chỉ có một phiên bản đào tạo). Các mạng nơ-ron này rõ ràng là không độc lập vì chúng chia sẻ nhiều trọng số của chúng, nhưng chúng hoàn toàn khác nhau. Mạng nơ-ron kết quả có thể được coi là một tập hợp trung bình của tất cả các mạng nơ-ron nhỏ hơn này.

Để thực hiện dropout chương trình bằng Keras, bạn có thể sử dụng lớp `keras.layers.Dropout`. Trong quá trình đào tạo, nó sẽ bỏ ngẫu nhiên một số đầu vào (đặt chúng thành 0) và chia các đầu vào còn lại cho xác suất giữ. Sau khi đào tạo, nó không làm gì cả, nó chỉ chuyển đầu vào cho lớp tiếp theo. Ví dụ: mã sau đây áp dụng quy định bỏ trang trước mỗi Lớp dày đặc, sử dụng tỷ lệ bỏ học là 0,2:

```
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
```

Nếu bạn quan sát thấy rằng mô hình đang bị overfitting, bạn có thể tăng tỷ lệ bỏ học. Ngược lại, bạn nên thử giảm tỷ lệ bỏ học nếu mô hình phù hợp với bộ đào tạo. Nó cũng có thể giúp tăng tỷ lệ bỏ học cho các lớp lớn và giảm tỷ lệ này cho các lớp nhỏ. Hơn nữa, nhiều kiến trúc hiện đại chỉ sử dụng tính năng bỏ qua sau lớp ẩn cuối cùng, vì vậy bạn có thể muốn thử điều này nếu tỷ lệ bỏ hoàn toàn quá mạnh.

Dropout có xu hướng làm chậm sự hội tụ đáng kể, nhưng nó thường dẫn đến một mô hình tốt hơn nhiều khi được điều chỉnh đúng cách. Vì vậy, nó thường rất đáng để bỏ thêm thời gian và nỗ lực.

### Monte-Carlo (MC) Dropout
Vào năm 2016, một bài báo (*"Dropout as a Bayesian Approximation: Representing Model Uncertainty in Deep Learning"*) của Yarin Gal và Zoubin Ghahramani đã bổ sung thêm nhiều lý do chính đáng để bỏ học:
- Đầu tiên, bài báo thiết lập một kết nối sâu sắc giữa các mạng dropout (tức là mạng nơ-ron chứa lớp bỏ học trước mỗi lớp trọng số) và suy luận gần đúng của Bayes, đưa ra sự bỏ học là một biện minh toán học chắc chắn.
- Thứ hai, họ giới thiệu một kỹ thuật mạnh mẽ được gọi là MC Dropout, có thể tăng hiệu suất của bất kỳ mô hình bỏ học được đào tạo nào mà không cần phải đào tạo lại hoặc thậm chí sửa đổi nó!
- Hơn nữa, MC Dropout cũng cung cấp một thước đo tốt hơn nhiều về độ không chắc chắn của mô hình.
- Cuối cùng, nó cũng rất đơn giản để thực hiện. Nếu tất cả điều này nghe giống như một quảng cáo "một trò lừa kỳ lạ", thì hãy xem đoạn mã sau. Đó là việc triển khai đầy đủ MC Dropout, thúc đẩy mô hình bỏ học mà chúng tôi đã đào tạo trước đó, mà không cần đào tạo lại:
```
with keras.backend.learning_phase_scope(1): # force training mode = dropout on
    y_probas = np.stack([model.predict(X_test_scaled)
                        for sample in range(100)])
y_proba = y_probas.mean(axis=0)
```
Nói tóm lại, MC Dropout là một kỹ thuật tuyệt vời giúp tăng cường các mô hình dropout và cung cấp các ước tính về độ không chắc chắn tốt hơn. Và tất nhiên, vì nó chỉ là học sinh bỏ học thường xuyên trong quá trình đào tạo, nó cũng hoạt động như một bộ điều chỉnh.

### Max-Norm Regularization
Một kỹ thuật chính quy khác khá phổ biến đối với mạng nơ-ron được gọi là chính quy hóa tối đa: đối với mỗi nơ-ron, nó ràng buộc trọng số $w$ của các kết nối đến sao cho $∥*w*∥_2 ≤ _r_$, trong đó $r$ là siêu thông số tối đa và $∥ · ∥_2$ là định mức $ℓ_2$.

Max-norm regularization không thêm thuật ngữ regularization loss vào hàm tổn thất tổng thể. Thay vào đó, nó thường được thực hiện bằng tính toán ∥w∥2 sau mỗi bước huấn luyện và cắt bớt $w$ nếu cần ($w ← w \frac{r}{∥ w ∥_2}$).

Giảm r làm tăng lượng chính quy hóa và giúp giảm trang bị quá mức. Việc điều chỉnh Maxnorm cũng có thể giúp giảm bớt các vấn đề về gradient biến mất / bùng nổ (nếu bạn không sử dụng Batch Normalization).

Để triển khai max-norm regularization trong Keras, chỉ cần đặt mọi đối số `kernel_constraint` của lớp ẩn thành ràng buộc `max_norm()`, với giá trị tối đa thích hợp, ví dụ:
```
keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",
                    kernel_constraint=keras.constraints.max_norm(1.))
```

Sau mỗi lần lặp lại huấn luyện, phương thức `fit()` của model sẽ gọi đối tượng được trả về bởi `max_norm()`, chuyển cho nó trọng lượng của lớp và nhận lại trọng lượng bị cắt bớt, sau đó sẽ thay thế trọng lượng của lớp. Như chúng ta sẽ thấy trong Chương 12, bạn có thể xác định hàm ràng buộc tùy chỉnh của riêng mình nếu bạn cần và sử dụng nó làm `kernel_constraint`. Bạn cũng có thể hạn chế các điều khoản thiên vị bằng cách đặt đối số `bias_con_straint`.

Hàm `max_norm()` có đối số là trục mặc định là 0. Lớp dày đặc thường có trọng số là hình dạng [số đầu vào, số tế bào thần kinh], do đó, sử dụng `axis=0` có nghĩa là giới hạn định mức tối đa sẽ áp dụng độc lập cho trọng số của từng nơ-ron vectơ. Nếu bạn muốn sử dụng max-norm với các lớp chập (xem Chương 14), hãy đảm bảo đặt đối số trục của ràng buộc `max_norm()` một cách thích hợp (thường là `axis=[0, 1, 2]`)

## 11.5 Tóm tắt và Hướng dẫn Thực hành
Trong chương này, chúng tôi đề cập đến một loạt các kỹ thuật và bạn có thể tự hỏi mình nên sử dụng kỹ thuật nào. Cấu hình trong Bảng 11-2 sẽ hoạt động tốt trong hầu hết các trường hợp, mà không yêu cầu điều chỉnh siêu thông số nhiều.

*Bảng 11.2 Cấu hình DNN mặc định*

| Hyperparameter    	| Default value 	|
|-	|-	|
| Kernel initializer: 	| LeCun initialization 	|
| Activation function: 	| SELU 	|
| Normalization: 	| None (self-normalization) 	|
| Regularization: 	| Early stopping 	|
|  Optimizer:	|  Nadam	|
| Optimizer: 	| Performance scheduling 	|

Đừng quên chuẩn hóa các tính năng đầu vào! Tất nhiên, bạn cũng nên cố gắng sử dụng lại các phần của mạng nơ-ron được đào tạo trước nếu bạn có thể tìm thấy mạng giải quyết vấn đề tương tự hoặc sử dụng đào tạo trước không được giám sát nếu bạn có nhiều dữ liệu chưa được gắn nhãn hoặc đào tạo trước trên một tác vụ phụ trợ nếu bạn có nhiều dữ liệu được gắn nhãn cho một tác vụ tương tự.

1. Nếu mô hình của bạn tự chuẩn hóa (self-normalizes):
- Nếu nó phù hợp với tập huấn luyện, thì bạn nên thêm tính năng alpha dropout(và luôn sử dụng tính năng dừng sớm). Không sử dụng các phương pháp chính quy hóa khác, nếu không chúng sẽ phá vỡ quá trình tự chuẩn hóa.
2. Nếu mô hình của bạn không thể tự chuẩn hóa (ví dụ: đó là một mạng lặp lại hoặc nó chứa các kết nối bỏ qua):
- Bạn có thể thử sử dụng ELU (hoặc một chức năng kích hoạt khác) thay vì SELU, nó có thể hoạt động tốt hơn. Đảm bảo thay đổi phương thức khởi tạo cho phù hợp (ví dụ: He init cho ELU hoặc ReLU).
- Nếu đó là một mạng sâu, bạn nên sử dụng Batch Normalization sau mỗi lớp ẩn. Nếu nó phù hợp với bộ đào tạo, bạn cũng có thể thử sử dụng định mức tối đa hoặc chính quy ℓ2.
3. Nếu bạn cần một mô hình thưa thớt, bạn có thể sử dụng chính quy ℓ1 (và tùy chọn loại bỏ các trọng lượng nhỏ sau khi đào tạo). Nếu bạn cần một mô hình thậm chí còn thưa thớt hơn, bạn có thể thử sử dụng FTRL thay vì tối ưu hóa Nadam, cùng với chính thức hóa ℓ1. Trong mọi trường hợp, điều này sẽ phá vỡ quá trình tự bình thường hóa, vì vậy bạn sẽ cần chuyển sang BN nếu mô hình của bạn có chiều sâu.
4. Nếu bạn cần một mô hình có độ trễ thấp (một mô hình thực hiện các dự đoán nhanh như chớp), bạn có thể cần sử dụng ít lớp hơn, tránh Chuẩn hóa hàng loạt và có thể thay thế chức năng kích hoạt SELU bằng ReLU bị rò rỉ. Có một mô hình thưa thớt cũng sẽ hữu ích. Bạn cũng có thể muốn giảm độ chính xác float từ 32 bit xuống 16 bit (hoặc thậm chí 8 bit)
5. Nếu bạn đang xây dựng một ứng dụng nhạy cảm với rủi ro hoặc độ trễ suy luận không quá quan trọng trong ứng dụng của bạn, bạn có thể sử dụng MC Dropout để tăng hiệu suất và nhận được các ước tính xác suất đáng tin cậy hơn, cùng với các ước tính không chắc chắn.

Với những hướng dẫn này, bây giờ bạn đã sẵn sàng để huấn luyện lưới rất sâu! Tôi hy vọng bây giờ bạn tin chắc rằng bạn có thể đi một chặng đường rất dài chỉ khi sử dụng Keras. Tuy nhiên, có thể có lúc bạn cần phải kiểm soát nhiều hơn nữa, chẳng hạn như để viết một hàm mất mát tùy chỉnh hoặc để tinh chỉnh thuật toán đào tạo. Đối với những trường hợp như vậy, bạn sẽ cần sử dụng API cấp thấp hơn của TensorFlow, như chúng ta sẽ thấy trong chương tiếp theo.